# HOW TO RUN A TELEGRAM BOT IN COLAB

In [3]:
%%writefile requirements.txt

python-telegram-bot==12.4.2
tornado==4.5.3
ipykernel
scipy==1.1.0
torchvision
pillow==4.1.1


Overwriting requirements.txt


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 368kB 7.5MB/s 
     |████████████████████████████████| 31.2MB 101kB/s 
     |████████████████████████████████| 5.7MB 33.9MB/s 
     |████████████████████████████████| 2.3MB 38.9MB/s 
     |████████████████████████████████| 112kB 44.7MB/s 
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35417 sha256=2899d94ca55a03427707b7beb06fccee05b8f8ecb3406a80785b20ab96ccdea0
  Stored in directory: /root/.cache/pip/wheels/4b/f4/11/bc4166107c27f07fd7bba707ffcb439619197638a1ac986df3
Successfully built olefile
ERROR: scikit-image 0.16.2 has requirement pillow>=4.3.0, but you'll have pillow 4.1.1 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: python-telegram-bot 12.4.2 has requirement tornado>=5.1, but you'll have tornado 4.5.3 which 

In [0]:
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from scipy import misc
import copy
import matplotlib.pyplot as plt
%matplotlib inline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
class ContentLoss(nn.Module):
        def __init__(self, target,):
            super().__init__()
            self.target = target.detach()
            self.loss = F.mse_loss(self.target, self.target)

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

def gram_matrix(input):
        batch_size, h, w, f_map_num = input.size()  
        features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t()) 

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(batch_size * h * w * f_map_num)

class StyleLoss(nn.Module):
        def __init__(self, target_feature):
            super().__init__()
            self.target = gram_matrix(target_feature).detach()
            self.loss = F.mse_loss(self.target, self.target)

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

class Normalization(nn.Module):
        def __init__(self, mean, std):
            super().__init__()
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            return (img - self.mean) / self.std


In [0]:
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                                   style_img, content_img,
                                   content_layers,
                                   style_layers):
        cnn = copy.deepcopy(cnn)
        normalization = Normalization(normalization_mean, normalization_std).to(device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = f'conv_{i}'
            elif isinstance(layer, nn.ReLU):
                name = f'relu_{i}'
                # The in-place version doesn't play very nicely with the ContentLoss
                # and StyleLoss we insert below. So we replace with out-of-place
                # ones here.
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = f'pool_{i}'
            elif isinstance(layer, nn.BatchNorm2d):
                name = f'bn_{i}'
            else:
                raise RuntimeError(f'Unrecognized layer: {layer.__class__.__name__}')

            model.add_module(name, layer)

            if name in content_layers:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module(f"content_loss_{i}", content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module(f"style_loss_{i}", style_loss)
                style_losses.append(style_loss)

        # now we trim off the layers after the last content and style losses
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses

def get_input_optimizer(input_img):
        # this line to show that input is a parameter that requires a gradient
        optimizer = optim.LBFGS([input_img.requires_grad_()]) 
        return optimizer



In [0]:
class StyleTransferModel:
    def __init__(self):
        self.content_layers_default = ['conv_4']
        self.style_layers_default = ['conv_1','conv_2', 'conv_3', 'conv_4', 'conv_5']
        self.normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
        self.normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)
        self.cnn = models.vgg19(pretrained=True).features.to(device).eval()
        
    def transfer_style(self, content_img_stream, style_img_stream, 
                       num_steps=500, style_weight=100000, content_weight=1):
        print(device)
        print('Building the style transfer model..')
        content_img = self.image_to_tensor(content_img_stream)
        style_img = self.image_to_tensor(style_img_stream)
        
        input_img = content_img.clone()

        model, style_losses, content_losses = get_style_model_and_losses(
            self.cnn,
            self.normalization_mean,
            self.normalization_std, 
            style_img, content_img,
            content_layers=self.content_layers_default,
            style_layers=self.style_layers_default
        )
        optimizer = get_input_optimizer(input_img)
        print('Optimizing..')
        run = [0]
        while run[0] <= num_steps:

            def closure():
                # correct the values 
                input_img.data.clamp_(0, 1)

                optimizer.zero_grad()

                model(input_img)

                style_score = 0
                content_score = 0

                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss
              
                style_score *= style_weight
                content_score *= content_weight

                loss = style_score + content_score
                loss.backward()

                run[0] += 1
                if run[0] % 50 == 0:
                    print(f"run {run}:")
                    print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                        style_score.item(), content_score.item()))
                    print()

                return style_score + content_score

            optimizer.step(closure)

        # a last correction...
        input_img.data.clamp_(0, 1)
        return self.tensor_to_image(input_img)

    @staticmethod
    def image_to_tensor(img_stream, imsize=512):
        loader = transforms.Compose([
        transforms.Resize(imsize), 
        transforms.CenterCrop(imsize),
        transforms.ToTensor()])  

        image = Image.open(img_stream)
        image = loader(image).unsqueeze(0)
        return image.to(device, torch.float)

    @staticmethod
    def tensor_to_image(tensor):
        unloader = transforms.ToPILImage()
        tensor = tensor.cpu().clone()   
        tensor = tensor.squeeze(0)     
        image = unloader(tensor)
        return misc.toimage(image)
    

In [0]:
from io import BytesIO
from telegram.ext import Updater, MessageHandler, CommandHandler, Filters
import logging

model = StyleTransferModel()
first_image_file = {}


def send_prediction_on_photo(bot, update):

    chat_id = update.message.chat_id
    print(f"Got image from {chat_id}")

    image_info = update.message.photo[-1]
    image_file = bot.get_file(image_info)
    
    if chat_id in first_image_file:
        # первая картинка, которая к нам пришла станет content image, а вторая style image
        content_image_stream = BytesIO()
        first_image_file[chat_id].download(out=content_image_stream)
        del first_image_file[chat_id]

        style_image_stream = BytesIO()
        image_file.download(out=style_image_stream)

        output = model.transfer_style(content_image_stream, style_image_stream, num_steps=200)

        # теперь отправим назад фото
        output_stream = BytesIO()
        output.save(output_stream, format='PNG')
        output_stream.seek(0)
        bot.send_photo(chat_id, photo=output_stream)
        print("Sent Photo to user")
    else:
        first_image_file[chat_id] = image_file

def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Hey, just send me two photos!')

    

In [0]:
token = '1021525121:AAFLQ3QaBengPrKBIiGTZdfzHRE1g4-O1tE'
logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        level=logging.INFO)
updater = Updater(token=token,  request_kwargs={'proxy_url': 'socks5h://163.172.152.192:1080'})

updater.dispatcher.add_handler(CommandHandler('start', startCommand))
updater.dispatcher.add_handler(MessageHandler(Filters.photo, send_prediction_on_photo))
updater.start_polling(clean=True)
updater.idle()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """


Got image from 82969866
Got image from 82969866
cuda
Building the style transfer model..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Optimizing..
run [50]:
Style Loss : 18.505602 Content Loss: 12.675617

run [100]:
Style Loss : 5.982433 Content Loss: 11.752635

run [150]:
Style Loss : 2.445985 Content Loss: 11.218088

